In [1]:
import random
import pandas as pd
from Bio import Align
from Bio.Align import substitution_matrices

AMINO_ACIDS = 'ACDEFGHIKLMNPQRSTVWY'
# BLOSUM62 matrix
blosum62 = {
    'A': {'A': 4, 'C': 0, 'D': -2, 'E': -1, 'F': -2, 'G': 0, 'H': -2, 'I': -1, 'K': -1, 'L': -1, 'M': -1, 'N': -2, 'P': -1, 'Q': -1, 'R': -1, 'S': 1, 'T': 0, 'V': 0, 'W': -3, 'Y': -2},
    'C': {'A': 0, 'C': 9, 'D': -3, 'E': -4, 'F': -2, 'G': -3, 'H': -3, 'I': -1, 'K': -3, 'L': -1, 'M': -1, 'N': -3, 'P': -3, 'Q': -3, 'R': -3, 'S': -1, 'T': -1, 'V': -1, 'W': -2, 'Y': -2},
    'D': {'A': -2, 'C': -3, 'D': 6, 'E': 2, 'F': -3, 'G': -1, 'H': -1, 'I': -3, 'K': -1, 'L': -4, 'M': -3, 'N': 1, 'P': -1, 'Q': 0, 'R': -2, 'S': 0, 'T': -1, 'V': -3, 'W': -4, 'Y': -3},
    'E': {'A': -1, 'C': -4, 'D': 2, 'E': 5, 'F': -3, 'G': -2, 'H': 0, 'I': -3, 'K': 1, 'L': -3, 'M': -2, 'N': 0, 'P': -1, 'Q': 2, 'R': 0, 'S': 0, 'T': -1, 'V': -2, 'W': -3, 'Y': -2},
    'F': {'A': -2, 'C': -2, 'D': -3, 'E': -3, 'F': 6, 'G': -3, 'H': -1, 'I': 0, 'K': -3, 'L': 0, 'M': 0, 'N': -3, 'P': -4, 'Q': -3, 'R': -3, 'S': -2, 'T': -2, 'V': -1, 'W': 1, 'Y': 3},
    'G': {'A': 0, 'C': -3, 'D': -1, 'E': -2, 'F': -3, 'G': 6, 'H': -2, 'I': -4, 'K': -2, 'L': -4, 'M': -3, 'N': 0, 'P': -2, 'Q': -2, 'R': -2, 'S': 0, 'T': -2, 'V': -3, 'W': -2, 'Y': -3},
    'H': {'A': -2, 'C': -3, 'D': -1, 'E': 0, 'F': -1, 'G': -2, 'H': 8, 'I': -3, 'K': -1, 'L': -3, 'M': -2, 'N': 1, 'P': -2, 'Q': 0, 'R': 0, 'S': -1, 'T': -2, 'V': -3, 'W': -2, 'Y': 2},
    'I': {'A': -1, 'C': -1, 'D': -3, 'E': -3, 'F': 0, 'G': -4, 'H': -3, 'I': 4, 'K': -3, 'L': 2, 'M': 1, 'N': -3, 'P': -3, 'Q': -3, 'R': -3, 'S': -2, 'T': -1, 'V': 3, 'W': -3, 'Y': -1},
    'K': {'A': -1, 'C': -3, 'D': -1, 'E': 1, 'F': -3, 'G': -2, 'H': -1, 'I': -3, 'K': 5, 'L': -2, 'M': -1, 'N': 0, 'P': -1, 'Q': 1, 'R': 2, 'S': 0, 'T': -1, 'V': -2, 'W': -3, 'Y': -2},
    'L': {'A': -1, 'C': -1, 'D': -4, 'E': -3, 'F': 0, 'G': -4, 'H': -3, 'I': 2, 'K': -2, 'L': 4, 'M': 2, 'N': -3, 'P': -3, 'Q': -2, 'R': -2, 'S': -2, 'T': -1, 'V': 1, 'W': -2, 'Y': -1},
    'M': {'A': -1, 'C': -1, 'D': -3, 'E': -2, 'F': 0, 'G': -3, 'H': -2, 'I': 1, 'K': -1, 'L': 2, 'M': 5, 'N': -2, 'P': -2, 'Q': 0, 'R': -1, 'S': -1, 'T': -1, 'V': 1, 'W': -1, 'Y': -1},
    'N': {'A': -2, 'C': -3, 'D': 1, 'E': 0, 'F': -3, 'G': 0, 'H': 1, 'I': -3, 'K': 0, 'L': -3, 'M': -2, 'N': 6, 'P': -2, 'Q': 0, 'R': 0, 'S': 1, 'T': 0, 'V': -3, 'W': -4, 'Y': -2},
    'P': {'A': -1, 'C': -3, 'D': -1, 'E': -1, 'F': -4, 'G': -2, 'H': -2, 'I': -3, 'K': -1, 'L': -3, 'M': -2, 'N': -2, 'P': 7, 'Q': -1, 'R': -2, 'S': -1, 'T': -1, 'V': -2, 'W': -4, 'Y': -3},
    'Q': {'A': -1, 'C': -3, 'D': 0, 'E': 2, 'F': -3, 'G': -2, 'H': 0, 'I': -3, 'K': 1, 'L': -2, 'M': 0, 'N': 0, 'P': -1, 'Q': 5, 'R': 1, 'S': 0, 'T': -1, 'V': -2, 'W': -2, 'Y': -1},
    'R': {'A': -1, 'C': -3, 'D': -2, 'E': 0, 'F': -3, 'G': -2, 'H': 0, 'I': -3, 'K': 2, 'L': -2, 'M': -1, 'N': 0, 'P': -2, 'Q': 1, 'R': 5, 'S': -1, 'T': -1, 'V': -3, 'W': -3, 'Y': -2},
    'S': {'A': 1, 'C': -1, 'D': 0, 'E': 0, 'F': -2, 'G': 0, 'H': -1, 'I': -2, 'K': 0, 'L': -2, 'M': -1, 'N': 1, 'P': -1, 'Q': 0, 'R': -1, 'S': 4, 'T': 1, 'V': -2, 'W': -3, 'Y': -2},
    'T': {'A': 0, 'C': -1, 'D': -1, 'E': -1, 'F': -2, 'G': -2, 'H': -2, 'I': -1, 'K': -1, 'L': -1, 'M': -1, 'N': 0, 'P': -1, 'Q': -1, 'R': -1, 'S': 1, 'T': 5, 'V': 0, 'W': -2, 'Y': -2},
    'V': {'A': 0, 'C': -1, 'D': -3, 'E': -2, 'F': -1, 'G': -3, 'H': -3, 'I': 3, 'K': -2, 'L': 1, 'M': 1, 'N': -3, 'P': -2, 'Q': -2, 'R': -3, 'S': -2, 'T': 0, 'V': 4, 'W': -3, 'Y': -1},
    'W': {'A': -3, 'C': -2, 'D': -4, 'E': -3, 'F': 1, 'G': -2, 'H': -2, 'I': -3, 'K': -3, 'L': -2, 'M': -1, 'N': -4, 'P': -4, 'Q': -2, 'R': -3, 'S': -3, 'T': -2, 'V': -3, 'W': 11, 'Y': 2},
    'Y': {'A': -2, 'C': -2, 'D': -3, 'E': -2, 'F': 3, 'G': -3, 'H': 2, 'I': -1, 'K': -2, 'L': -1, 'M': -1, 'N': -2, 'P': -3, 'Q': -1, 'R': -2, 'S': -2, 'T': -2, 'V': -1, 'W': 2, 'Y': 7}
}
SEQUENCES_PATH = "./data/biolip_sequences.pkl"
GEN_NUM = 10

In [2]:
def calculate_alignment_score(seq1, seq2):
    """Calculate global alignment score based on BLOSUM62 matrix"""
    aligner = Align.PairwiseAligner(mode="global")
    aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
    score = 0
    for aa1, aa2 in zip(seq1, seq2):
        score += blosum62[aa1][aa2]  # Use a gap penalty of -4 if pair not found
    return score

def generate_seq(initial_sequence, num_iterations=10000, num_results=1):
    results = []
    initial_sequence = list(initial_sequence)
    for _ in range(num_iterations):
        new_sequence = ""
        for _ in initial_sequence:
            new_sequence+=random.choice(AMINO_ACIDS)
        mutated_score = calculate_alignment_score(initial_sequence, new_sequence)
        results.append({"seq":"".join(new_sequence), "score":mutated_score})
    unique_sequences = set()
    unique_sorted_sequences = []
    for seq in results:
        sequence_key = seq['seq']
        if sequence_key not in unique_sequences:
            unique_sequences.add(sequence_key)
            unique_sorted_sequences.append(seq)
    unique_sorted_sequences.sort(key=lambda x: x['score'], reverse=True)    
    return unique_sorted_sequences[:num_results]

# Example usage:
initial_sequence = "ARNDC"
num_results = GEN_NUM

best_sequences = generate_seq(initial_sequence, num_results=num_results)
print(f"Initial sequence: {initial_sequence}")
print(f"Top {num_results} sequences found:")
for idx, v in enumerate(best_sequences, start=1):
    print(f"{idx}. Sequence: {v['seq']}, Score: {v['score']}")


Initial sequence: ARNDC
Top 10 sequences found:
1. Sequence: SRNYC, Score: 18
2. Sequence: YRGDC, Score: 18
3. Sequence: RCNDC, Score: 17
4. Sequence: RKDDC, Score: 17
5. Sequence: ARYNC, Score: 17
6. Sequence: RRVDC, Score: 16
7. Sequence: DRMDC, Score: 16
8. Sequence: LKTDC, Score: 16
9. Sequence: REDDC, Score: 15
10. Sequence: RRDNC, Score: 15


In [3]:
df = pd.read_pickle(SEQUENCES_PATH)
df.ligand_sequence

7         PPRPLPVAPGSSKT
17             TPYDINQML
18              VPLRPMTY
19             KPIVQYDNF
23          NGDFEEIPEEYL
              ...       
36597      ISARKYPSDWEEW
36603         QPISVTMVTP
36605          QPISVTMVT
36634       TSKFPHLTFESP
36635    MLSEGYLSGLEYWND
Name: ligand_sequence, Length: 4758, dtype: object

In [4]:
res = []
for idx, seq in enumerate(df.ligand_sequence):
    gen_sequences = generate_seq(seq,num_results=GEN_NUM)
    res.append([s["seq"] for s in gen_sequences])
    # verbose
    print(f"Generating {idx}, seq:{seq}")
    for idx, v in enumerate(gen_sequences, start=1):
        print(f"{idx}. Sequence: {v['seq']}, Score: {v['score']}")
    print("")

Generating 0, seq:PPRPLPVAPGSSKT
1. Sequence: PPYPYPMAQIQGCD, Score: 21
2. Sequence: PFSEAGLVPGDEKC, Score: 16
3. Sequence: PPRSFEIKRTPNQN, Score: 16
4. Sequence: PGPTYPYCPNQEER, Score: 14
5. Sequence: EAEPRPNAPQHHAA, Score: 13
6. Sequence: KTCRMPDAPGLPNE, Score: 12
7. Sequence: PFSKYPTVVGSMAI, Score: 12
8. Sequence: PFWPCSCTPGFRPP, Score: 12
9. Sequence: SIAPWTPTPGDFAT, Score: 12
10. Sequence: DPRACIATPKEWKR, Score: 12

Generating 1, seq:TPYDINQML
1. Sequence: TPYTCNQYG, Score: 23
2. Sequence: IPYDVKDTV, Score: 22
3. Sequence: TPYPLTRLQ, Score: 21
4. Sequence: VPNDVHMLA, Score: 16
5. Sequence: VPHEQDQMH, Score: 16
6. Sequence: KVIDISQQL, Score: 16
7. Sequence: THYNFWHML, Score: 16
8. Sequence: TPMWIDEAI, Score: 15
9. Sequence: TLHEAEQVL, Score: 15
10. Sequence: EPYDWTHQA, Score: 15

Generating 2, seq:VPLRPMTY
1. Sequence: YPIKPISF, Score: 22
2. Sequence: IPHNPYPY, Score: 19
3. Sequence: EPKRPATV, Score: 18
4. Sequence: NPLRNQQY, Score: 17
5. Sequence: MPPSHLTY, Score: 16
6. Sequence: 

In [5]:
df["generated_seq"] = res
df.head()

,pdb_id,receptor_chain,ligand_chain,binding_site_sequence,binding_site_pos,receptor_sequence,ligand_sequence,generated_seq
7,1a0n,B,A,YYWPNY,"[8, 10, 36, 51, 53, 54]",GSTGVTLFVALYDYEARTEDDLSFHKGEKFQILNSSEGDWWEARSL...,PPRPLPVAPGSSKT,"[PPYPYPMAQIQGCD, PFSEAGLVPGDEKC, PPRSFEIKRTPNQ..."
17,1a1m,A,C,YYNYNYRYDYTKLYWY,"[7, 9, 63, 74, 77, 84, 97, 99, 114, 123, 143, ...",GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,TPYDINQML,"[TPYTCNQYG, IPYDVKDTV, TPYPLTRLQ, VPNDVHMLA, V..."
18,1a1n,A,C,RNIFTTYSNYRYSTWVLYW,"[62, 63, 66, 67, 69, 73, 74, 77, 80, 84, 97, 9...",GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,VPLRPMTY,"[YPIKPISF, IPHNPYPY, EPKRPATV, NPLRNQQY, MPPSH..."
19,1a1o,A,C,YRNITNIYRYYTQYWY,"[7, 62, 63, 66, 69, 77, 80, 84, 97, 99, 123, 1...",GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,KPIVQYDNF,"[EPITLYVGF, KPSKDMDTF, NPCDYYDFF, EPLKMYPRY, K..."
23,1a2c,H,I,FQLRTRYKI,"[19, 24, 60, 68, 69, 70, 71, 77, 78]",IVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLL...,NGDFEEIPEEYL,"[GGHYNRTPNRMM, NQNKEPGSNEYM, KKRFEANPISYV, NGE..."


In [6]:
(df.generated_seq.apply(len)!=GEN_NUM).sum()

0

In [7]:
df.to_pickle(f"blosum_generated_{GEN_NUM}.pkl")